In [ ]:
!pip3 install opencv-python
!pip3 install ultralytics
!pip3 install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 24.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.8/782.8 kB 8.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import boto3


s3_client = boto3.client(
    service_name='s3',
    region_name='us-west-2',
    aws_access_key_id='XXXX',
    aws_secret_access_key='XXXX'
)

# Print out bucket names
# for bucket in s3_client.buckets.all():
#     print(bucket.name)

def upload_file( bucket_name , file_path , object_name):
    print("upload file function")
    try:
        s3_client.upload_file(file_path , bucket_name ,object_name)
        return True
    except Exception as e:
        print(e)
        return False
    
def download_file( bucket_name , file_path , object_name):
    try:
        s3_client.download_file( bucket_name ,object_name,file_path)
        return True
    except Exception as e:
        print(e)
        return False
    
def delete_file(bucket_name , file_name):
    try:
        s3_client.delete_object(Bucket= bucket_name , Key=file_name)
        return True
    except Exception as e:
        print(e)
        return False


In [ ]:
download_file( 'demo-test-pipeline' , 'ss_2.mp4' , 'ss_2.mp4')

True

In [ ]:
print("filler stuff")

filler stuff


In [ ]:
from ultralytics import YOLO
import cv2 
import sys

model = YOLO('yolov8n.pt')


def detector_model(image):
    results = model.predict([image], verbose=False , conf = 0.3)  

    cls_names = model.names
    person = 0
    vehicle = 0
    # Iterate over the predictions
    for i, result in enumerate(results):
        # Extract boxes, scores and class labels
        bboxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        cls = result.boxes.cls.cpu().numpy()
        
        # Iterate over the bboxes, filter by confidence and draw over the image
        for j in range(len(bboxes)):
            (x1, y1, x2, y2), score, c = bboxes[j], scores[j], cls[j]
            if int(c)==0:
                person = person + 1
            elif int(c) in [1,2,3,4,5,6,7,8]:
                vehicle = vehicle + 1
            if score >= 0.3:
                cv2.putText(image, f"{cls_names[c]} {score:.4f}",(int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (int(c == 0) * 255, int(c == 1) * 255, int(c ==2) * 255), 2)

        # Save image with detections
    return image , person , vehicle

In [ ]:
import cv2
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, IntegerType, BinaryType

# Initialize Spark session
spark = SparkSession.builder.appName("VideoToDataFrame").getOrCreate()

# Define schema for the DataFrame
schema = StructType([
    StructField("frame_id", IntegerType(), True),
    StructField("frame_data", BinaryType(), True),
    StructField("person_count", IntegerType(), True),
    StructField("vehicle_count", IntegerType(), True)
])

# Define function to read frames from video
def read_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame , person , vehicle = detector_model(frame)
        ret, buffer = cv2.imencode('.jpg', frame)
        frames.append((frame_id, buffer.tobytes(),person , vehicle))
        frame_id += 1
        if frame_id ==300:
            break
    cap.release()
    return frames

# Create an RDD from the video frames
video_path = "ss_2.mp4"
#"/Volumes/trial_workspace/default/test/ss_2.mp4"
frames_rdd = spark.sparkContext.parallelize(read_video_frames(video_path))

# Convert RDD to DataFrame
df = frames_rdd.toDF(schema)
df.show()

+--------+--------------------+------------+-------------+
|frame_id|          frame_data|person_count|vehicle_count|
+--------+--------------------+------------+-------------+
|       0|[FF D8 FF E0 00 1...|           1|           18|
|       1|[FF D8 FF E0 00 1...|           2|           17|
|       2|[FF D8 FF E0 00 1...|           2|           16|
|       3|[FF D8 FF E0 00 1...|           3|           17|
|       4|[FF D8 FF E0 00 1...|           3|           21|
|       5|[FF D8 FF E0 00 1...|           3|           20|
|       6|[FF D8 FF E0 00 1...|           2|           17|
|       7|[FF D8 FF E0 00 1...|           2|           19|
|       8|[FF D8 FF E0 00 1...|           3|           20|
|       9|[FF D8 FF E0 00 1...|           2|           16|
|      10|[FF D8 FF E0 00 1...|           1|           16|
|      11|[FF D8 FF E0 00 1...|           2|           19|
|      12|[FF D8 FF E0 00 1...|           1|           17|
|      13|[FF D8 FF E0 00 1...|           1|           1

In [ ]:
ls

demo*  err.txt*  meta_info.csv*  output_video.mp4*  ss_2.mp4*  yolov8n.pt*


In [ ]:
df1 = df.collect()
#df1[0].__getitem__('frame_data') 
nparr = np.frombuffer(df1[0].__getitem__('frame_data'), np.uint8)
frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
height, width  = 480 , 640  #, _ = frame.shape
video_writer = cv2.VideoWriter("output_video.mp4", 
                                cv2.VideoWriter_fourcc(*'mp4v'), 
                                20, 
                                (width,height))
meta_info = []
for i in range(len(df1)):
    nparr = np.frombuffer(df1[i].__getitem__('frame_data'), np.uint8)
    frame_id , p_count , v_count = [df1[i].__getitem__(j) for j in ['frame_id', 'person_count','vehicle_count']]
    meta_info.append([frame_id , p_count , v_count])
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    #print(frame.shape)
    frame= cv2.resize(frame,(width,height))
    video_writer.write(frame)
video_writer.release()

In [ ]:
upload_file( 'demo-test-pipeline' , 'output_video.mp4' , 'output_video.mp4')

upload file function


True

In [ ]:
#!rm -r output_video.mp4

In [ ]:
import pandas as pd 

df_pd = pd.DataFrame(meta_info , columns = ['frame_id', 'person_count','vehicle_count'])

In [ ]:
df_pd.to_csv('meta_info.csv',index=False)

In [ ]:
ls

demo*  err.txt*  meta_info.csv*  output_video.mp4*  ss_2.mp4*  yolov8n.pt*


In [ ]:
upload_file( 'demo-test-pipeline' , 'meta_info.csv', 'meta_info.csv')
#upload_file( 'demo-test-pipeline' , 'meta_info.csv', 'output_folder/meta_info.csv' ) #bucket_name , file_path , object_name)

upload file function


True

In [ ]:
!pip install snowflake-connector-python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.5 MB ? eta -:--:--
     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/2.5 MB 4.5 MB/s eta 0:00:01
     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/2.5 MB 7.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/2.5 MB 9.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━ 1.5/2.5 MB 10.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 2.2/2.5 MB 12.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/105.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 21.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas
conn = snow.connect(
    user= 'prathik30',
    password= 'XXXXX',
    account= 'XXXXX',
    warehouse= 'compute_wh',
    database= 'DW_COURSE_DB',
    schema= 'VIDEO_STREAM',
)
cur = conn.cursor()
df_pd.columns = ['FRAME_ID', 'PERSON_COUNT', 'VEHICLE_COUNT']
print(df_pd.head(10))
write_pandas(conn, df_pd, "STREAMCOUNT",overwrite=True)


   FRAME_ID  PERSON_COUNT  VEHICLE_COUNT
0         0             1             18
1         1             2             17
2         2             2             16
3         3             3             17
4         4             3             21
5         5             3             20
6         6             2             17
7         7             2             19
8         8             3             20
9         9             2             16


(True,
 1,
 300,
 [('wyqexiykrm/file0.txt', 'LOADED', 300, 300, 1, 0, None, None, None, None)])

In [ ]:
cur.close()
conn.close()